# Relatório de Fotos ANTT - CCR

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import re

In [2]:
def convert_string(string: str) -> str:
    if '_' in string:
        # Convert snake_case to camelCase
        return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), string)
    else:
        # Convert camelCase to snake_case
        return re.sub(r'([A-Z])', lambda x: '_' + x.group(1).lower(), string)

In [3]:
def ret_campo_foto_array(classe):
    campos=[]
    inner_campos=[]
    for a in classe.form_fields['fields']:
        if a['dataType'] == 'arrayOfObjects':
            temp = {}
            temp[convert_string(a['apiName'])] = []
            for inner in a['innerFields']:
                if inner['dataType'] == 'innerImagesArray':
                    temp[convert_string(a['apiName'])].append({"main_array_display_name":a['displayName'],
                                               "inner_api":convert_string(inner['apiName']),
                                               "inner_display":inner['displayName']})
                    inner_campos.append({convert_string(a['apiName']):convert_string(inner['apiName'])})
            campos.append(temp)
    return campos,inner_campos

In [4]:
def concat_uuid_inner_image(classe,rep):
    items=ret_campo_foto_array(classe)[1]
    counter=[]
    for a in items:
        for k,v in a.items():
            if rep.form_data.get(k):
                for b in rep.form_data.get(k):
                    if b.get(v):
                        counter += b.get(v)
    return counter

In [5]:
company=Company.objects.get(name='CCR - Rio Sp')

In [6]:
filename='RelImagens'

In [7]:
pd.DataFrame().to_excel('RelImagens.xlsx')

In [8]:
reps=Reporting.objects.filter(company=company,occurrence_type__name="Monitoração de Terraplenos e Estruturas de Contenção")
reps.count(),company

(3846, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [9]:
dados=[]
contention_esperada,croqui_esperada,pano_esperada,pato_esperada,drainage_pic_esperada,detail_esperada,cris_esperada,rig_esperada,lef_esperada,foot_esperada=1,1,1,1,1,1,1,1,1,1
for index,a in enumerate(tqdm(reps)):
    contention_size,primeira_imagem_croqui_geral,panoramic_size,patology_size,details_size,criste_size,right_size,left_size,foot_size,drainage_pic_size,therapy_pic_size,img_antt=0,0,0,0,0,0,0,0,0,0,0,0
    rel = 'Não OK'
    
    if a.form_data.get('therapy') and len(a.form_data.get('therapy')) >0:
        therapy_pic = a.form_data.get('therapy')[0]
        therapy_pic_size = len(therapy_pic.get('treatment_images')) if therapy_pic.get('treatment_images') else 0
    if a.form_data.get('drainage_pictures') and len(a.form_data.get('drainage_pictures')) >0:
        drainage_pic = a.form_data.get('drainage_pictures')[0]
        drainage_pic_size = len(drainage_pic.get('drainage_picture')) if drainage_pic.get('drainage_picture') else 0
    if a.form_data.get('contention') and len(a.form_data.get('contention')) >0:
        contention_pictures=a.form_data['contention'][0]
        contention_size=len(contention_pictures.get('inner_pictures')) if contention_pictures.get('inner_pictures') else 0
    if a.form_data.get('mandatory_pictures') and len(a.form_data.get('mandatory_pictures')) >0:
#         print(index)
        mandatory_pictures=a.form_data['mandatory_pictures'][0]
        panoramic_size=len(mandatory_pictures.get('panoramic_mandatory')) if mandatory_pictures.get('panoramic_mandatory') else 0
        patology_size=len(mandatory_pictures.get('patology_mandatory')) if mandatory_pictures.get('patology_mandatory') else 0
        details_size=len(mandatory_pictures.get('details_mandatory')) if mandatory_pictures.get('details_mandatory') else 0
        criste_size=len(mandatory_pictures.get('criste_mandatory')) if mandatory_pictures.get('criste_mandatory') else 0
        right_size=len(mandatory_pictures.get('right_mandatory')) if mandatory_pictures.get('right_mandatory') else 0
        left_size=len(mandatory_pictures.get('left_mandatory')) if mandatory_pictures.get('left_mandatory') else 0
        foot_size=len(mandatory_pictures.get('foot_mandatory')) if mandatory_pictures.get('foot_mandatory') else 0
        
        #     A última que sairá na exportação no formato validado, foto do campo 
        #     “Croqui Atual”, será a primeira foto não obrigatória inserida no 
        #     apontamento.
        lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
        croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi).order_by("-uploaded_at")
        img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
        primeira_imagem_croqui_geral=croqui.count() if croqui.exists() else 0
        
    if panoramic_size > 0:
        if primeira_imagem_croqui_geral > 0: # “Croqui Atual”, será a primeira foto não obrigatória inserida no apontamento.
            if contention_size > 0: # foto da contenção > foto da patologia > foto da drenagem
                if patology_size > 0 or drainage_pic_size > 0:
                    rel = 'OK'
            elif patology_size > 0:
                if drainage_pic_size > 0:
                    rel = 'OK'
            elif drainage_pic_size > 0:
                if foot_size > 0 or criste_size > 0 or left_size > 0 or right_size > 0 or details_size > 0:
                    rel = 'OK'
            if foot_size > 0: # regra de prioridade:pé > crista > vista esquerda > vista direita > detalhe.
                if criste_size > 0 or left_size > 0 or right_size > 0 or details_size > 0:
                    rel = 'OK'
            elif criste_size > 0:
                if left_size > 0 or right_size > 0 or details_size > 0:
                    rel = 'OK'
            elif left_size > 0:
                if right_size > 0 or details_size > 0:
                    rel = 'OK'
            elif right_size > 0:
                if details_size > 0:
                    rel = 'OK'
            if panoramic_size > 2: # feito com segundas e terceiras fotos panorâmicas
                rel = 'OK'
            if therapy_pic_size > 1: # terceira e quarta fotos na exportação deverá ser feito com as fotos de anomalias
                 rel = 'OK'
            if primeira_imagem_croqui_geral > 2:
                rel = 'OK'
            
    if rel == 'OK' and img_antt < 4:
        rel = 'Não OK'
        
    dados.append([a.number,
                  contention_size,contention_esperada,
                  primeira_imagem_croqui_geral, croqui_esperada,
                  panoramic_size, pano_esperada,
                  patology_size,pato_esperada,
                  details_size,detail_esperada,
                  criste_size,cris_esperada,
                  right_size,rig_esperada,
                  left_size,lef_esperada,
                  foot_size,foot_esperada,
                  drainage_pic_size,drainage_pic_esperada,
                  img_antt,rel])

  0%|          | 0/3846 [00:00<?, ?it/s]

In [10]:
df_terraplenos=pd.DataFrame(data=dados,columns=['Serial',
                                    'Contenção Adicionada','Contenção Esperada',
                                    'Foto sem classificação Adiconada','Foto sem classificação Esperada',
                                    'Panoramica Atual Adicionada','Panoramica Atual Esperada',
                                    'Patologia Adicionada','Patologia Atual Esperada',
                                    'Detalhes Adicionada','Detalhes Atual Esperada',
                                    'Crista Adicionada','Crista Atual Esperada',
                                    'Vista Direita Adicionada','Vista Direita Atual Esperada',
                                    'Vista Esquerda Adicionada','Vista Esquerda Atual Esperada',
                                    'Pé Adicionada','Pé Atual Esperada',
                                    'Drenagem Adicionada','Drenagem Esperada',
                                    'Fotos compartilhadas com ANTT',
                                    'Relatório OK'])
df_terraplenos.set_index("Serial", inplace=True)

In [11]:
df_terraplenos

,Contenção Adicionada,Contenção Esperada,Foto sem classificação Adiconada,Foto sem classificação Esperada,Panoramica Atual Adicionada,Panoramica Atual Esperada,Patologia Adicionada,Patologia Atual Esperada,Detalhes Adicionada,Detalhes Atual Esperada,...,Vista Direita Adicionada,Vista Direita Atual Esperada,Vista Esquerda Adicionada,Vista Esquerda Atual Esperada,Pé Adicionada,Pé Atual Esperada,Drenagem Adicionada,Drenagem Esperada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,,,,,,,,,,,,,,,,,,
CCR-TER-2023.03471,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,Não OK
CCR-TER-2023.03494,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,Não OK
CCR-TER-2023.03517,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,Não OK
CCR-TER-2023.03521,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,Não OK
CCR-TER-2023.03557,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,Não OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCR-TER-2023.05119,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,Não OK
CCR-TER-2023.05120,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,Não OK
CCR-TER-2023.05121,0,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,0,1,0,Não OK


In [12]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df_terraplenos.to_excel(writer, sheet_name='Mon. Terraplenos', index=True)

### df para Monitoração de Barreira New Jersey/Defensas Em OAE/Tela Antiofuscante
Cada um precisa de duas fotos obrigatórias innerArray: 
Início
Fim

In [13]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Monitoração de Barreira New Jersey",
                                                                        "Monitoração de Defensas Em OAE",
                                                                        "Monitoração de Tela Antiofuscante"])
reps.count(),company

(3494, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [14]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Monitoração de Barreira New Jersey'}, {'occurrence_type__name': 'Monitoração de Defensas Em OAE'}, {'occurrence_type__name': 'Monitoração de Tela Antiofuscante'}]>

In [15]:
print(ret_campo_foto_array(reps[0].occurrence_type)[1])

[{'therapy': 'treatment_images'}, {'mandatory_pictures': 'foto_inicio'}, {'mandatory_pictures': 'foto_fim'}]


In [16]:
ini_esp,fim_esp = 1,1
dados2=[]
for index,a in enumerate(tqdm(reps)):
    ini_size,fim_size,img_antt=0,0,0
    rel = 'Não OK'
    
    if a.form_data.get('mandatory_pictures') and len(a.form_data.get('mandatory_pictures')) >0:
#         print(index)
        for b in a.form_data['mandatory_pictures']:
            mandatory_pictures=b
            ini_size+=len(mandatory_pictures.get('foto_inicio')) if mandatory_pictures.get('foto_inicio') else 0
            fim_size+=len(mandatory_pictures.get('foto_fim')) if mandatory_pictures.get('foto_fim') else 0
    # Foto geral do apontamento tirando as do innerArray      
    lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
    croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi)
    foto_geral=croqui.count() if croqui.exists() else 0
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    if ini_size > 0 and fim_size > 0:
        rel = 'OK'
    if rel == 'OK' and img_antt < 2:
        rel = 'Não OK'
    
    dados2.append([a.number,
                  ini_size,ini_esp,
                 fim_size,fim_esp,foto_geral,img_antt,rel])

  0%|          | 0/3494 [00:00<?, ?it/s]

In [17]:
df2=pd.DataFrame(data=dados2,columns=['Serial',
                                    'Foto Inicio Adicionada','Foto Inicio Esperada',
                                    'Foto Fim Adicionada','Foto Fim Esperada','Foto sem classificação',
                                    'Fotos compartilhadas com ANTT','Relatório OK'])
df2.set_index("Serial", inplace=True)

In [18]:
df2

,Foto Inicio Adicionada,Foto Inicio Esperada,Foto Fim Adicionada,Foto Fim Esperada,Foto sem classificação,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,,,,
CCR-EPS-2023.19006,1,1,1,1,0,0,Não OK
CCR-EPS-2023.02446,0,1,0,1,2,0,Não OK
CCR-EPS-2023.02448,0,1,0,1,2,0,Não OK
CCR-EPS-2023.02451,0,1,0,1,2,0,Não OK
CCR-EPS-2023.02450,0,1,0,1,2,0,Não OK
...,...,...,...,...,...,...,...
CCR-EPS-2023.02801,0,1,0,1,2,0,Não OK
CCR-EPS-2023.02812,0,1,0,1,2,0,Não OK
CCR-EPS-2023.02823,0,1,0,1,2,0,Não OK


In [19]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df2.to_excel(writer, sheet_name='Mon. New-OAE-Anti', index=True)

# df para Monitoração de Acessos; Monitoração de Defensas Metálicas
2 fotos do campo de imagens usual foto geral 

In [20]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Monitoração de Acessos",
                                                                        "Monitoração de Defensas Metálicas"])
reps.count(),company

(8754, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [21]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Monitoração de Acessos'}, {'occurrence_type__name': 'Monitoração de Defensas Metálicas'}]>

In [22]:
foto_esp = 2
dados3=[]
for index,a in enumerate(tqdm(reps)):
    foto_geral=0
    rel = 'Não OK'
    
    lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
    croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi)
    foto_geral=croqui.count() if croqui.exists() else 0
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    if foto_geral > 1:
        rel = 'OK'
    if rel == 'OK' and img_antt < 2:
        rel = 'Não OK'
    
    dados3.append([a.number,
                  foto_geral,foto_esp,
                   img_antt,rel])

  0%|          | 0/8754 [00:00<?, ?it/s]

In [23]:
df3=pd.DataFrame(data=dados3,columns=['Serial',
                                    'Foto sem classificação Adicionada','Foto sem classificação Esperada',
                                    'Fotos compartilhadas com ANTT','Relatório OK'])
df3.set_index("Serial", inplace=True)

In [24]:
df3

,Foto sem classificação Adicionada,Foto sem classificação Esperada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,
CCR-FDO-2023.04622,2,2,0,Não OK
CCR-FDO-2023.04625,2,2,0,Não OK
CCR-FDO-2023.04628,2,2,0,Não OK
CCR-FDO-2023.04632,2,2,0,Não OK
CCR-FDO-2023.04631,2,2,0,Não OK
...,...,...,...,...
CCR-EPS-2023.19009,2,2,0,Não OK
CCR-EPS-2023.18899,2,2,0,Não OK
CCR-EPS-2023.19000,3,2,0,Não OK


In [25]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df3.to_excel(writer, sheet_name='Mon. Acessos-Defensa', index=True)

# Monitoração de Ocupações
2 fotos do campo usual de imagens. 
1 foto do campo Foto do Croqui (inner). 

In [26]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Monitoração de Ocupações"])
reps.count(),company

(2050, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [27]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Monitoração de Ocupações'}]>

In [28]:
foto_esp,croqui_esp = 2,1
dados4=[]
for index,a in enumerate(tqdm(reps)):
    foto_geral,croqui_size=0,0
    rel = 'Não OK'
    
    if a.form_data.get('imagens_croqui_faixa_dominio') and len(a.form_data.get('imagens_croqui_faixa_dominio')) >0:
        for b in a.form_data.get('imagens_croqui_faixa_dominio'):
            mandatory_pictures=b
            croqui_size+=len(mandatory_pictures.get('imagens_croqui')) if mandatory_pictures.get('imagens_croqui') else 0
    lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
    croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi)
    foto_geral=croqui.count() if croqui.exists() else 0
    
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    if foto_geral > 1 and croqui_size > 0:
        rel = 'OK'
    if rel == 'OK' and img_antt < 2:
        rel = 'Não OK'
    
    dados4.append([a.number,
                  foto_geral,foto_esp,
                  croqui_size,croqui_esp,
                  img_antt,rel])

  0%|          | 0/2050 [00:00<?, ?it/s]

In [29]:
df4=pd.DataFrame(data=dados4,columns=['Serial',
                                    'Foto sem classificação Adicionada','Foto sem classificação Esperada',
                                     'Croqui Adicionada','Croqui Esperada',
                                     'Fotos compartilhadas com ANTT','Relatório OK'])
df4.set_index("Serial", inplace=True)

In [30]:
df4

,Foto sem classificação Adicionada,Foto sem classificação Esperada,Croqui Adicionada,Croqui Esperada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,,,
CCR-FDO-2023.01113,0,2,0,1,0,Não OK
CCR-FDO-2023.01122,0,2,0,1,0,Não OK
CCR-FDO-2023.01333,0,2,0,1,0,Não OK
CCR-FDO-2023.01641,0,2,0,1,0,Não OK
CCR-FDO-2023.01798,0,2,0,1,0,Não OK
...,...,...,...,...,...,...
CCR-FDO-2023.03147,0,2,0,1,0,Não OK
CCR-FDO-2023.03146,0,2,0,1,0,Não OK
CCR-FDO-2023.03145,0,2,0,1,0,Não OK


In [31]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df4.to_excel(writer, sheet_name='Mon. Ocupacoes', index=True)

# Monitoração de Sinalização Vertical
1 para 1 apontamento 

In [6]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Monitoração de Sinalização Vertical"]
                             ).only('occurrence_type','uuid')
reps.count(),company

(23372, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [8]:
foto_esp = 1
dados5=[]
for index,a in enumerate(tqdm(reps)):
    foto_geral=0
    
    lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
    croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi)
    foto_geral=croqui.count() if croqui.exists() else 0
    
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    if foto_geral > 0:
        rel = 'OK'
    if rel == 'OK' and img_antt < 1:
        rel = 'Não OK'
    
    dados5.append([a.number,
                  foto_geral,foto_esp,
                  img_antt,rel])

  0%|          | 0/23372 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df5=pd.DataFrame(data=dados5,columns=['Serial',
                                    'Foto sem classificação Adicionada','Foto sem classificação Esperada',
                                     'Fotos compartilhadas com ANTT','Relatório OK'])
df5.set_index("Serial", inplace=True)

In [ ]:
df5

In [ ]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df5.to_excel(writer, sheet_name='Mon. Sin.Vertical', index=True)

# Retrorrefletância Horizontal Longitudinal
Depende do número estação ativadas. Para cada apontamento necessário   contar número de estações será 1 Foto para 1 estação ativada

In [32]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Retrorrefletância Horizontal Longitudinal"])
reps.count(),company

(150, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [33]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Retrorrefletância Horizontal Longitudinal'}]>

In [34]:
ret_campo_foto_array(reps[0].occurrence_type)[1]

[{'station_pictures_one': 'foto_da_estacao_um'},
 {'station_pictures_two': 'foto_da_estacao_dois'},
 {'station_pictures_three': 'foto_da_estacao_tres'}]

In [35]:
dados6=[]
for index,a in enumerate(tqdm(reps)):
    total,total_estacoes=0,0
    rel = 'Não OK'
    
    temp={y[1]:0 for z in ret_campo_foto_array(reps[0].occurrence_type)[1] for y in z.items()}
    for x in ret_campo_foto_array(a.occurrence_type)[1]:
        for k,v in x.items():
            if a.form_data.get(k) and len(a.form_data.get(k)) >0:
                for i,b in enumerate(a.form_data.get(k)):
                    mandatory_pictures=b
                    temp[v]+=len(mandatory_pictures.get(v)) if mandatory_pictures.get(v) else 0
                    
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    if a.form_data.get('enable_station_one'):           
        total+=1
    if a.form_data.get('enable_station_two'):           
        total+=1
    if a.form_data.get('enable_station_three'):           
        total+=1
    
    total_estacoes = temp.get('foto_da_estacao_um')+temp.get('foto_da_estacao_dois')+temp.get('foto_da_estacao_tres')
    
    if total_estacoes >= total and img_antt >= total:
        rel = 'OK'
    
    temp['Serial'] = a.number
    temp['Fotos esperada'] = total
    temp['Fotos compartilhadas com ANTT'] = img_antt
    temp['Relatório OK'] = rel

    dados6.append(temp)

  0%|          | 0/150 [00:00<?, ?it/s]

In [36]:
df6=pd.DataFrame(data=dados6
                )
df6.set_index("Serial", inplace=True)

In [37]:
df6 = df6.rename(columns={'foto_da_estacao_um': 'Estação 1', 
                          'foto_da_estacao_dois': 'Estação 2', 
                          'foto_da_estacao_tres': 'Estação 3'})
df6

,Estação 1,Estação 2,Estação 3,Fotos esperada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,,,
CCR-DEF-2023.03630,1,1,1,3,0,Não OK
CCR-DEF-2023.03637,1,1,1,3,0,Não OK
CCR-DEF-2023.03633,1,1,1,3,0,Não OK
CCR-DEF-2023.03571,1,1,1,3,0,Não OK
CCR-DEF-2023.03652,1,1,1,3,0,Não OK
...,...,...,...,...,...,...
CCR-DEF-2023.03661,2,2,2,3,0,Não OK
CCR-DEF-2023.03649,1,1,1,3,0,Não OK
CCR-DEF-2023.03645,1,1,1,3,0,Não OK


In [38]:
df6.loc[['CCR-DEF-2023.03637','CCR-DEF-2023.03630'],['Estação 1','Estação 2']]

,Estação 1,Estação 2
Serial,,
CCR-DEF-2023.03637,1,1
CCR-DEF-2023.03630,1,1


In [39]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df6.to_excel(writer, sheet_name='Retro Hor.Long.', index=True)

# Retrorrefletância Horizontal Dispositivos
Depende do número estação ativadas e Ramos que constituem o relatório. Para cada apontamento necessário contar número de estações será 1 Foto para 1 estação ativada

In [40]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Retrorrefletância de Dispositivos"])
reps.count(),company

(529, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [41]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Retrorrefletância de Dispositivos'}]>

In [42]:
ret_campo_foto_array(reps[0].occurrence_type)[1]

[{'station_one': 'station_image'},
 {'station_two': 'station_image_two'},
 {'station_three': 'station_image_third'}]

In [43]:
station_spec = 1
dados7=[]
for index,a in enumerate(tqdm(reps)):
    foto_geral,total=0,0
    rel = 'Não OK'
    
    temp={y[1]:0 for z in ret_campo_foto_array(reps[0].occurrence_type)[1] for y in z.items()}
    for x in ret_campo_foto_array(a.occurrence_type)[1]:
        for k,v in x.items():
            if a.form_data.get(k) and len(a.form_data.get(k)) >0:
                for i,b in enumerate(a.form_data.get(k)):
                    mandatory_pictures=b
                    temp[v]+=len(mandatory_pictures.get(v)) if mandatory_pictures.get(v) else 0
                    
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    if a.form_data.get('enable_station_one'):           
        total+=1
    if a.form_data.get('enable_station_two'):           
        total+=1
    if a.form_data.get('enable_station_three'):           
        total+=1
    
    total_estacoes = temp.get('station_image')+temp.get('station_image_two')+temp.get('station_image_third')
    
    if total_estacoes >= total and img_antt >= total:
        rel = 'OK'
    
    items = list(temp.items())
    items.insert(0, ('Serial', a.number))
    resultado = dict(items)
    resultado['Fotos esperada'] = total
    resultado['Fotos compartilhadas com ANTT'] = img_antt
    resultado['Relatório OK'] = rel
#     print(resultado)
    dados7.append(resultado)

  0%|          | 0/529 [00:00<?, ?it/s]

In [44]:
df7=pd.DataFrame(data=dados7
                )
df7.set_index("Serial", inplace=True)
df7 = df7.rename(columns={'station_image': 'Estação 1', 
                          'station_image_two': 'Estação 2', 
                          'station_image_third': 'Estação 3'})

In [45]:
df7

,Estação 1,Estação 2,Estação 3,Fotos esperada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,,,
CCR-DEF-2023.03232,1,0,0,1,0,Não OK
CCR-DEF-2023.03252,1,0,0,1,0,Não OK
CCR-DEF-2023.03241,1,0,0,1,0,Não OK
CCR-DEF-2023.03235,1,0,0,1,0,Não OK
CCR-DEF-2023.03236,1,0,0,1,0,Não OK
...,...,...,...,...,...,...
CCR-DEF-2023.03188,1,0,0,1,0,Não OK
CCR-DEF-2023.03191,1,0,0,1,0,Não OK
CCR-DEF-2023.03192,1,0,0,1,0,Não OK


In [46]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df7.to_excel(writer, sheet_name='Retro Dispositivos', index=True)

# Retrorrefletância Horizontal Zebrados
2 para 1 apontamento  (imagem repetida)

In [47]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Retrorrefletância Horizontal de Zebrado"])
reps.count(),company

(2031, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [48]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Retrorrefletância Horizontal de Zebrado'}]>

In [49]:
foto_esp = 1
dados8=[]
for index,a in enumerate(tqdm(reps)):
    foto_geral=0
    rel='Não OK'
    
    lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
    croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi)
    foto_geral=croqui.count() if croqui.exists() else 0
    
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    if foto_geral > 0 and img_antt > 0:
        rel = 'OK'
    
    dados8.append([a.number,
                  foto_geral,foto_esp,
                  img_antt,rel])

  0%|          | 0/2031 [00:00<?, ?it/s]

In [50]:
df8=pd.DataFrame(data=dados8,columns=['Serial',
                                    'Foto sem classificação Adicionada','Foto sem classificação Esperada',
                                     'Fotos compartilhadas com ANTT','Relatório OK'])
df8.set_index("Serial", inplace=True)

In [51]:
df8

,Foto sem classificação Adicionada,Foto sem classificação Esperada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,
CCR-DEF-2023.00944,1,1,0,Não OK
CCR-DEF-2023.00945,1,1,0,Não OK
CCR-DEF-2023.00982,1,1,0,Não OK
CCR-DEF-2023.01031,1,1,0,Não OK
CCR-DEF-2023.01036,1,1,0,Não OK
...,...,...,...,...
CCR-DEF-2023.01943,1,1,0,Não OK
CCR-DEF-2023.01944,1,1,0,Não OK
CCR-DEF-2023.01945,1,1,0,Não OK


In [52]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df8.to_excel(writer, sheet_name='Retro Zebrados', index=True)

# Retrorrefletância Horizontal Marcas Viárias
Todas com - 1° imagem  Arquivos e imagens {Foto} 

In [53]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Retrorrefletância de Marcas Viárias"])
reps.count(),company

(652, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [54]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Retrorrefletância de Marcas Viárias'}]>

In [55]:
foto_esp = 1
dados9=[]
for index,a in enumerate(tqdm(reps)):
    foto_geral=0
    rel = 'Não OK'
    
    lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
    croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi)
    foto_geral=croqui.count() if croqui.exists() else 0
    
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    if foto_geral > 0 and img_antt > 0:
        rel = 'OK'
        
    dados9.append([a.number,
                  foto_geral,foto_esp,
                  img_antt,rel])

  0%|          | 0/652 [00:00<?, ?it/s]

In [56]:
df9=pd.DataFrame(data=dados9,columns=['Serial',
                                    'Foto sem classificação Adicionada','Foto sem classificação Esperada',
                                     'Fotos compartilhadas com ANTT','Relatório OK'])
df9.set_index("Serial", inplace=True)

In [57]:
df9

,Foto sem classificação Adicionada,Foto sem classificação Esperada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,
CCR-DEF-2023.02532,1,1,0,Não OK
CCR-DEF-2023.02537,1,1,0,Não OK
CCR-DEF-2023.02552,1,1,0,Não OK
CCR-DEF-2023.02560,1,1,0,Não OK
CCR-DEF-2023.02566,1,1,0,Não OK
...,...,...,...,...
CCR-DEF-2023.32788,0,1,0,Não OK
CCR-DEF-2023.32789,0,1,0,Não OK
CCR-DEF-2023.32790,0,1,0,Não OK


In [58]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df9.to_excel(writer, sheet_name='Retro Marcas Viarias', index=True)

# Monitoração de Instalações Elétricas de Edificações
2 imagem para 1 apontamento . Se houver anomalia adicionada 1 imagem por anomalia

In [59]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Monitoração de Instalações Elétricas de Edificações"])
reps.count(),company

(44, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [60]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Monitoração de Instalações Elétricas de Edificações'}]>

In [61]:
ret_campo_foto_array(reps[0].occurrence_type)[1]

[{'therapy': 'treatment_images'}]

In [62]:
dados10=[]
for index,a in enumerate(tqdm(reps)):
    treatment_size,foto_geral=0,0
    foto_geral_esp,foto_esp = 2,0
    
    rel = 'Não OK'
    
    if a.form_data.get('therapy') and len(a.form_data.get('therapy')) > 0:
        foto_esp = len(a.form_data.get('therapy'))
        for b in a.form_data.get('therapy'):
            mandatory_pictures=b
            treatment_size+=len(mandatory_pictures.get('treatment_images')) if mandatory_pictures.get('treatment_images') else 0
    
    # Foto geral do apontamento tirando as do innerArray      
    lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
    croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi)
    foto_geral=croqui.count() if croqui.exists() else 0
    
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    foto_esp = foto_esp if treatment_size == 0 else len(a.form_data.get('therapy'))
    
    if treatment_size >= foto_esp and img_antt >= foto_esp+foto_geral_esp:
        rel="OK"
    
    dados10.append([a.number,foto_geral,foto_geral_esp,
                  treatment_size,foto_esp,
                   img_antt,rel])

  0%|          | 0/44 [00:00<?, ?it/s]

In [63]:
df10=pd.DataFrame(data=dados10,columns=["Serial",'Foto sem classificação Adicionada','Foto sem classificação Esperada',
                                        "Foto Anomalia",'Fotos Anomalia Esperada',
                                       'Fotos compartilhadas com ANTT','Relatório OK'])
df10.set_index("Serial", inplace=True)

In [64]:
df10

,Foto sem classificação Adicionada,Foto sem classificação Esperada,Foto Anomalia,Fotos Anomalia Esperada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,,,
CCR-DEF-2023.33092,5,2,0,0,0,Não OK
CCR-DEF-2023.33098,2,2,0,0,0,Não OK
CCR-DEF-2023.33100,1,2,0,0,0,Não OK
CCR-DEF-2023.33104,2,2,0,0,0,Não OK
CCR-DEF-2023.33102,0,2,0,0,0,Não OK
CCR-DEF-2023.33105,2,2,0,0,0,Não OK
CCR-DEF-2023.33110,4,2,0,0,0,Não OK
CCR-DEF-2023.33108,2,2,0,0,0,Não OK
CCR-DEF-2023.33103,2,2,0,0,0,Não OK


In [65]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df10.to_excel(writer, sheet_name='Mon. Inst.Eletricas', index=True)

# Monitoração de Iluminação
4 imagem para 1 apontamento . Se houver anomalia adicionada 2 imagem por anomalia

In [66]:
reps=Reporting.objects.filter(company=company,occurrence_type__name__in=["Monitoração de Iluminação"])
reps.count(),company

(288, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>)

In [67]:
reps.values('occurrence_type__name').distinct()

<QuerySet [{'occurrence_type__name': 'Monitoração de Iluminação'}]>

In [68]:
ret_campo_foto_array(reps[0].occurrence_type)[1]

[{'therapy': 'treatment_images'},
 {'fotos_monitoracao': 'imagem_diurna'},
 {'fotos_monitoracao': 'imagem_noturna'}]

In [69]:
reps[0].occurrence_type.form_fields['fields']

[{'id': 1,
  'apiName': 'idCcrAntt',
  'dataType': 'string',
  'disableIf': True,
  'displayName': 'Identificação ANTT'},
 {'id': 201,
  'apiName': 'index',
  'dataType': 'number',
  'displayName': 'Índice (Serial Antt)'},
 {'id': 2,
  'apiName': 'latitude',
  'dataType': 'float',
  'displayName': 'Latitude'},
 {'id': 200,
  'apiName': 'latitudeEnd',
  'dataType': 'float',
  'displayName': 'Latitude (Fim)'},
 {'id': 3,
  'apiName': 'longitude',
  'dataType': 'float',
  'displayName': 'Longitude'},
 {'id': 300,
  'apiName': 'longitudeEnd',
  'dataType': 'float',
  'displayName': 'Longitude (Fim)'},
 {'id': 4,
  'apiName': 'city',
  'dataType': 'string',
  'disableIf': True,
  'displayName': 'Município'},
 {'id': 5,
  'apiName': 'uf',
  'dataType': 'string',
  'disableIf': True,
  'displayName': 'UF/Estado'},
 {'id': 6,
  'apiName': 'tipoiluminacao',
  'dataType': 'select',
  'displayName': 'Tipo de Iluminação',
  'selectOptions': {'options': [{'name': 'PASSARELA DE PEDESTRE',
     'valu

In [70]:
dados11=[]
for index,a in enumerate(tqdm(reps)):
    treatment_size,imagem_diurna,imagem_noturna=0,0,0
    treatment_size_esp,imagem_diurna_esp,imagem_noturna_esp = 0,2,2
    
    rel="Não OK"
    
    if a.form_data.get('therapy') and len(a.form_data.get('therapy')) > 0:
        treatment_size_esp = len(a.form_data.get('therapy'))*2
        for b in a.form_data.get('therapy'):
            mandatory_pictures=b
            treatment_size+=len(mandatory_pictures.get('treatment_images')) if mandatory_pictures.get('treatment_images') else 0
    if a.form_data.get('fotos_monitoracao') and len(a.form_data.get('fotos_monitoracao')) > 0:
        for b in a.form_data.get('fotos_monitoracao'):
            mandatory_pictures=b
            imagem_diurna+=len(mandatory_pictures.get('imagem_diurna')) if mandatory_pictures.get('imagem_diurna') else 0
    if a.form_data.get('fotos_monitoracao') and len(a.form_data.get('fotos_monitoracao')) > 0:
        for b in a.form_data.get('fotos_monitoracao'):
            mandatory_pictures=b
            imagem_noturna+=len(mandatory_pictures.get('imagem_noturna')) if mandatory_pictures.get('imagem_noturna') else 0       
    
    # Foto geral do apontamento tirando as do innerArray      
    lista_img_especi=concat_uuid_inner_image(a.occurrence_type,a)
    croqui = ReportingFile.objects.filter(reporting=a.uuid).exclude(uuid__in=lista_img_especi)
    foto_geral=croqui.count() if croqui.exists() else 0
    img_antt = ReportingFile.objects.filter(reporting=a.uuid,is_shared=True).count()
    
    treatment_size_esp = treatment_size_esp if treatment_size == 0 else len(a.form_data.get('therapy'))*2
    
    if treatment_size >= treatment_size_esp and imagem_diurna >= imagem_diurna_esp and imagem_noturna >= imagem_noturna_esp and img_antt >= imagem_diurna_esp+imagem_noturna_esp+treatment_size_esp:
        rel="OK"
    
    dados11.append([a.number,
                  imagem_diurna,imagem_diurna_esp,
                   imagem_noturna,imagem_noturna_esp,
                   treatment_size,treatment_size_esp,
                   foto_geral,
                   img_antt,rel])

  0%|          | 0/288 [00:00<?, ?it/s]

In [71]:
df11=pd.DataFrame(data=dados11,columns=["Serial",
                                        'Foto Diurna Adicionada','Foto Diurna Esperada',
                                        'Foto Noturna Adicionada','Foto Noturna Esperada',
                                        "Foto Anomalia Adicionada",'Fotos Anomalia Esperada',
                                       'Foto sem classificação Adicionada',
                                       'Fotos compartilhadas com ANTT','Relatório OK'])
df11.set_index("Serial", inplace=True)

In [72]:
df11

,Foto Diurna Adicionada,Foto Diurna Esperada,Foto Noturna Adicionada,Foto Noturna Esperada,Foto Anomalia Adicionada,Fotos Anomalia Esperada,Foto sem classificação Adicionada,Fotos compartilhadas com ANTT,Relatório OK
Serial,,,,,,,,,
CCR-DEF-2023.23612,0,2,3,2,0,0,0,0,Não OK
CCR-DEF-2023.23613,0,2,3,2,0,0,0,0,Não OK
CCR-DEF-2023.23772,0,2,0,2,0,0,0,0,Não OK
CCR-DEF-2023.23775,0,2,0,2,0,0,0,0,Não OK
CCR-DEF-2023.23710,2,2,0,2,0,0,0,0,Não OK
...,...,...,...,...,...,...,...,...,...
CCR-DEF-2023.23817,2,2,1,2,0,0,1,0,Não OK
CCR-DEF-2023.23676,1,2,1,2,0,0,0,0,Não OK
CCR-DEF-2023.23682,2,2,1,2,0,0,0,0,Não OK


In [73]:
with pd.ExcelWriter(filename+'.xlsx', mode='a',engine="openpyxl") as writer:
    df11.to_excel(writer, sheet_name='Mon. Iluminacoes', index=True)

# Deleta fotos que não existem nos arrayOfObjects

In [7]:
# from helpers.histories import bulk_update_with_history
# from copy import deepcopy
# from tqdm.notebook import tqdm

In [8]:
# company=Company.objects.get(name='CCR - Rio Sp')

In [9]:
# rep_edit=[]

# for a in tqdm(Reporting.objects.filter(company=company,
#                                        occurrence_type__name__in=["Retrorrefletância Horizontal Longitudinal",
#                                                                  "Retrorrefletância de Dispositivos",
#                                                                  "Retrorrefletância Horizontal de Zebrado",
#                                                                  "Retrorrefletância de Marcas Viárias",
#                                                                  "Monitoração de Instalações Elétricas de Edificações",
#                                                                  "Monitoração de Iluminação"])):
#     rep_bkp=deepcopy(a)
#     #     print(ret_campo_foto_array(a.occurrence_type)[1])
#     for b in ret_campo_foto_array(a.occurrence_type)[1]:
#         for c,d in b.items():
#             # pega a chave do arrayOfObjects se existir
#             if a.form_data.get(c):
#                 # itera sob o arrayOfObjects e o enumera para posterior exclusão
#                 for i,e in enumerate(a.form_data.get(c)):
#                     # se existir a chave da imagem criada no arrayOfObjects e não for 0
#                     if e.get(d) is not None and len(e.get(d)) >0:
#                         # itera sob os uuid de cada imagem e verifica se os reportingFile existe se caso não existe exclui no except
#                         for f in e.get(d):
#                             try:
#                                 ReportingFile.objects.get(uuid=f).upload.url
#                             except:
#                                 # remove o uuid que não existe da lista do innerImageArray
#                                 e.get(d).remove(f)
#                                 # se estiver vazio a lista do innerImageArray exclui a chave
#                                 if len(e.get(d))==0:
#                                     del a.form_data[c][i][d]
#     if a.form_data != rep_bkp.form_data:
#         rep_edit.append(a)


  0%|          | 0/3694 [00:00<?, ?it/s]

In [9]:
# len(rep_edit),rep_edit[0].form_data,rep_edit[0].number

IndexError: list index out of range

In [10]:
# apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=20,user=User.objects.get(username='rlcs'))

In [39]:
# Reporting.objects.get(uuid='4fcefe10-555a-437f-b4b7-629cb5c54f5f').form_data['fotos_monitoracao'][0] == {}

True